In [1]:
import os
import re
import json
import tika
import nltk
import spacy
import string
import collections
import itertools
import pandas as pd
from tika import parser
from nltk.tokenize import word_tokenize
nlp = spacy.load("en_core_web_sm")

from nltk.stem import PorterStemmer 
lemmatizer = PorterStemmer()

from spacy.gold import biluo_tags_from_offsets

In [2]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [3]:
#Checking length of each set
# set_name = 'Training Set/' 
# for i in os.listdir(set_name):
#     parsed = parser.from_file(set_name+i)
#     print(i,len(parsed['content']))

In [3]:
training_set = pd.read_csv('ValidationSet.csv')
training_set.head()

,File Name,Aggrement Value,Aggrement Start Date,Aggrement End Date,Renewal Notice (Days),Party One,Party Two
0,24158401-Rental-Agreement,12000,01.04.2008,31.03.2009,60.0,Hanumaiah,Vishal Bhardwaj
1,63793679-Rental-Agreement,9000,01.09.2011,31.08.2012,NaN,S Parthasarathy,Hari Kiran Tholeti
2,95980236-Rental-Agreement,9000,01.04.2010,31.03.2011,30.0,S.Sakunthala,V.V.Ravi Kian
3,156155545-Rental-Agreement-Kns-Home,12000,15.12.2012,14.11.2013,30.0,V.K.NATARAJ,VYSHNAVI DAIRY SPECIALITIES Private Ltd
4,195231682-This-RENTAL-AGREEMENT-is-Made-and-Ex...,13000,06.04.2013,05.03.2014,30.0,C.BHAGYAMMA,JP INTERIO


In [4]:
set_name = 'Training Set/' 
# dataframe = pd.DataFrame()
corpus = ""
for i in os.listdir(set_name)[1:]:
    parsed = parser.from_file(set_name+i)
    parsed['content'] = parsed['content'].replace('\n',' ')
    parsed['content'] = parsed['content'].replace('\t','')
#     cleaned_content = re.sub('[(]{1}[A-z\s]+[)]{1}',' ',parsed['content'])
#     cleaned_content = lemmatizer.stem(cleaned_content)
    corpus += parsed['content']
counts = collections.Counter(corpus.split())
most_common_words= [word.lower() for word, word_count in counts.most_common(50)]
most_common_words.remove('on') #on,and,month,by
most_common_words.remove('and')
most_common_words.remove('month')
most_common_words.remove('by')
most_common_words.remove('rent')
most_common_words.remove('pay')
most_common_words.remove('agrees')
most_common_words.append('rental')
most_common_words.append('agreement')
most_common_words.append('house')
most_common_words.remove('(')
most_common_words.remove(')')



In [7]:
set_name = 'Validation Set/' 
dataframe = pd.DataFrame()
total_match = 0
name_match = 0
all_sentences = []
for i in os.listdir(set_name):
    
    parsed = parser.from_file(set_name+i)
    parsed['content'] = parsed['content'].replace('\n',' ')
    parsed['content'] = parsed['content'].replace('\t','')
    file_name = os.path.splitext(os.path.splitext(i)[0])[0]
    print(file_name)
    if file_name in training_set['File Name'].values:
        total_match += 1
        cleaned_content = word_tokenize(parsed['content'])
        cleaned_content = " ".join(cleaned_content)
#         print(bcolors.OKBLUE,'cleaned_content:',bcolors.ENDC,cleaned_content)
        cleaned_content = re.sub('[(]{1}[A-z\s]+[)]{1}',' ',cleaned_content)
#         print(bcolors.OKBLUE,'cleaned_content:',bcolors.ENDC,cleaned_content)
        cleaned_content = re.sub('[!"#$%&\'()*+.,:;<=>?@[\\]^_`{|}~]',' ',cleaned_content)
        cleaned_content = re.sub('\sRs','',cleaned_content)
        encoded_string = cleaned_content.encode("ascii", "ignore")
        cleaned_content = encoded_string.decode()
        cleaned_content = ' '.join(cleaned_content.split())
#         print(bcolors.BOLD,'cleaned content:',bcolors.ENDC,cleaned_content)
        doc = nlp(cleaned_content)
        cleaned_content = list(doc.sents)
#         print(bcolors.BOLD,'cleaned content:',bcolors.ENDC,cleaned_content)
#         print(bcolors.OKGREEN,'cleaned content_1:',bcolors.ENDC,cleaned_content)
        dataframe = dataframe.append(pd.DataFrame({'filename':file_name,'content':cleaned_content}),ignore_index=True)
        for elements in cleaned_content:
            if len(str(elements).split())>2:
                all_sentences.append(elements)
        name_match += 1
        print('\n\n***************************\n\n')

156155545-Rental-Agreement-Kns-Home


***************************


195231682-This-RENTAL-AGREEMENT-is-Made-and-Executed-on-24th-Day-of-September


***************************


228094620-Rental-Agreement


***************************


239419594-Rental-Agreement


***************************


24158401-Rental-Agreement


***************************


269135973-Udaya-Rental-Agreement


***************************


63793679-Rental-Agreement


***************************


95980236-Rental-Agreement


***************************




In [8]:
all_sentences

[This deed of rental agreement is made on this day the 15th of December 2012 between V K NATARAJ,
 son of V KANDASWAMI CHETTIAR aged 55 years residing at,
 Door No - 5/8 F TYPE 4th MAIN ROAD SIDCO NAGAR,
 VILLIVAKKAM CHENNAI-600049 hereinafter called the LESSOR AND SRI VYSHNAVI,
 DAIRY SPECIALITIES Private Ltd,
 a company registered under the companies Act 1956 having its registered office at 170/IB M G R Nagar Manapakkam Village Chennai and Corporate Office at No 12/7 Sidco Nagar Villivakkam Chennai - 600049 Represented by its Chairman Mr RAJESH CHAVDA S/0 MANSHUK LAL CHAVDA aged about 56 years hereinafter called the LESSEE,
 Whereas LESSOR is the sole absolute owner of the land and building bearing door no 5/8 F TYPE 4th MAIN ROAD SIDCO NAGAR VILLIVAKKAM,
 both the lessor and lessee agree to the following terms and conditions NOW THIS DEED OF LEASE WITHNESSETH AS,
 FOLLOWS HERE -,
 The lessee agrees to pay a rent of 12 OOO on or before of 5th day of every English calendar month 2,
 A

In [142]:
## train_data=[]
for position,i in enumerate(all_sentences[1025:]):
    print(position+1025,i)
    inp = input('Whether entities ?')
    if inp == 'y':
        entities =[]
        nu_ent = int(input('Enter number of entities'))
        for rng in range(nu_ent):
            name_ent = input('Enter the entity')
            find=re.compile(name_ent)
            find.search(str(i)).span()
            type_ent = input('Enter entity type')
            tup = (find.search(str(i)).span()[0],find.search(str(i)).span()[1],type_ent)
            entities.append(tup)
        train_data.append((str(i),entities))

1025 This agreement made this 19 day of August 2010 between John Public of 1234 Any street Happytown Tennessee 23432 and Lagoon Landings LLC concerning the short tenn rental of the property located at 8730 Thomas Drive Unit 414 Panama City Beach Florida Total people in renting party 6 Adults 3 Children 3 not to exceed 8 Rental period begins at 3 pm on 8/17/2010 and ends at 10 am on 8/22/2010 Total Rental Amount 450 00 100 00 per night or 600 00 per week


KeyboardInterrupt: 

In [149]:
train_data = [('HOUSE RENTAL AGREEMENT Rental Agreement made on Jan 10 2011 between Namashivayam Plat No-182 Door',{'entities':
  [(48, 59, 'sdate'), (68, 80, 'fname')]}),
 ('East Street Kamaraj Nagar Thiruvanmiyur Chennai of the house and Mrs Thenmalar W/',{'entities':
  [(65, 78, 'sname')]}),
 ('Old 24th East Street Kamaraj Nagar Thiruvanmiyur along with electrical and sanitary fittings and other accessories fittings and structures from Jan 10 2011 at the monthly rent of 14500 being payable on or before 5th of every month to the Landlord',{'entities':
  [(144, 155, 'sdate'), (179, 184, 'rent')]}),
 ('The period of this agreement shall be twelve months w e f Jan 10 2011 2',{'entities':
  [(38, 44, 'dur'), (58, 69, 'sdate')]}),
 ('This rental agreement can be terminated at any time by three months notice on either side and on such termination the tenant shall surrender the vacant possession of the premises to the Landlord 5',{'entities':
  [(55, 60, 'notice')]}),
 ('This Tenancy Agreement is made and executed at Bangalore on this 22 01 2013 by MR P RAJA SEKHAR AGED ABOUT 28 YRS S/0',{'entities':
  [(65, 75, 'sdate'), (79, 95, 'fname')]}),
 ('BANGALORE-560 048 Hereinafter called the OWNER of the One Part AND MR VENKATA BHYRAVA MURTHY MUTNURI P/at NO 17-2-16 l/EERABHARAPURAM AGED ABOUT 26 YRSRAOAHMUNDRY ANDHRA PRADESH S',{'entities':
  [(67, 100, 'sname')]}),
 ('The tenancy shall commence from 11 04 2012 and initially valid for a period of eleven months and may be renewed with 5 enhanced rent for further periods before the expiry of this agreement after mutual consent between the parties 2',{'entities':
  [(32, 42, 'sdate'), (79, 85, 'dur')]}),
 ('The tenant shall pay a monthly rent of 15 000/- to the owner on or - before 5th of every succeeding month as per the english calendar month 3',{'entities':
  [(39, 47, 'rent')]}),
 ('The Owner/Tenant shall give to the Tenant/Cwner llU0 2 month advance notice for vacating of the premises 6',{'entities':
  [(53, 54, 'notice')]}),
 ('This agreement of Tenancy is made and executed at Chennai on 1st day of September 2011 1-3-2011 BETWEEN k Sarguneswaran s/o G Kasinath Rao aged about 54 years residing at No 20/3',{'entities':
  [(87, 95, 'sdate'), (104, 119, 'fname')]}),
 ('Velachery Chennai - 42 Tamil Nadu Hereinafter referred as the OWNER AND R Mathankumar s',{'entities':
  [(72, 85, 'sname')]}),
 ('The tenancy shall be in force for a period of eleven months from the above said date and the month of tenancy being the English calendar month 4 The Owner shall have the right to terminate the tenancy if the Tenant fails to pay the rent regularly for a consecutive period of two or more months or commits breach of any of the terms herein 5',{'entities':
  [(46, 52, 'dur')]}),
 ('It is hereby agreed that one month notice on either side is required for the termination of the tenancy 10 The tenancy shall be renewed for further period/',{'entities':
  [(25, 28, 'notice')]}),
 ('This RENTAL AGREEMENT is made and executed at Bangalore on this 5th day of April 2012 between Mrs PADMAVATI 154/2 1st floor 9th',{'entities':
  [(64, 85, 'sdate'), (94, 107, 'fname')]}),
 ('M Palya Bangalore - 560 075 hereinafter called the OWNER of one part Mrs SANGHAMITRA PARIDA Working at Sasken CommunicationTechnologies Ltd 139/25',{'entities':
  [(69, 91, 'sname')]}),
 ('The Tenant shall pay a monthly rent of 8000 on or before 5th of every month',{'entities':
  [(39, 43, 'rent')]}),
 ('The Tenant paid cash 60000/- Dated 05/04/2012 will be paid at time of occupation',{'entities':
  [(35, 45, 'sdate')]}),
 ('The Tenancy shall be in force for the period of 11 Months from the 5th April of 2012 month of tenancy being the English Calendar month 4',{'entities':
  [(48, 50, 'dur'), (67, 84, 'sdate')]}),
 ('It is hereby agreed that one-month notice on either side is required for the termination of tenancy 9',{'entities':
  [(25, 28, 'notice')]}),
 ('This DEED OF RENTAL AGREEMENT EXECUTED AT LANGKAWI KEDAH DARUL AMAN MALAYSIA ON 15 SEPTEMBER 2012 between ROHANI BINTI ISMAIL MALAYSIAN',{'entities':
  [(80, 97, 'sdate'), (106, 125, 'fname')]}),
 ('SECOND PARTY NAME NAME ADDRESS Hereafter called THE SECOND PARTY OF OTHER PART WITNESSETH as follows Whereas THE PREMISE - NO 73 A KG BUKIT NAU PADANG MATSIRAT 07100',{'entities':
  [(0, 12, 'sname')]}),
 ('The tenancy shall begin from 01 09 2012 and shall before a duration of 12 months',{'entities':
  [(29, 39, 'sdate'), (71, 80, 'dur')]}),
 ('The monthly rent shall be RM600-00 Per Month and payable by the SECOND PARTY during the course of FIRST WEEK of every month 3',{'entities':
  [(26, 34, 'rent')]}),
 ('electricity-water deposit RM500-00 and first monthly rental payment RM600-00',{'entities':
  [(68, 76, 'rent')]}),
 ('the SECOND PARTY desires to vacate the premises he should give notice of ONE calendar month in writing and similarly the FIRST PARTY should give ONE month notice in writing 10 1 -Signature of owner 1- Signature of witness',{'entities':
  [(73, 91, 'notice'), (145, 154, 'notice')]}),
 ('This deed of rental agreement executed at Bangalore this fifth day of December 2008 between MR K Kuttan S/',{'entities':
  [(57, 83, 'sdate'), (92, 103, 'fname')]}),
 ('Here after called the lessor which term shall where the contest so admits mean and includes his legal heirs representatives executors and assign and Sri P M Narayana Namboodri Laxmi Leela ground floor 3rd cross Ayyappa Nagar behind Ayyappa Temple Jalahalli West Bangalore - 15 referred as tenant which term shall wherever the context so admits mean and includes his legal heirs representations executors assign witnesses',{'entities':
  [(149, 181, 'sname')]}),
 ('and he has agreed to pay rent 4 000 00 for above',{'entities': [(30, 38, 'rent')]}),
 ('The agreement made only for eleven months from this date of agreement Fresh agreement with mutual terms and conditions have to be extended',{'entities':
  [(28, 34, 'dur')]}),
 ('Any arrears of rent electricity and water dues is permissible to deduct from the advance 4 Each party shall be liable to give three month advance notice for the termination of this agreement 5',{'entities':
  [(126, 131, 'notice')]}),
 ('This RENTAL AGREEMENT IS made and executed on 06th day of March 2013 at Hyderabad By and between Smt C BHAGYAMMA',{'entities':
  [(46, 68, 'sdate'), (97, 112, 'fname')]}),
 ('This tenancy is for a period of 11 months commencing from 06th day of April 2013 the date on which possession of the said flat is delivered and time to time it will be extended the tenancy period with consent of both parties',{'entities':
  [(32, 34, 'dur'), (58, 80, 'sdate')]}),
 ('The monthly Flat Rent is of RS 13 000/- per month and payable within the 10th day of same month 3',{'entities':
  [(28, 39, 'rent')]}),
 ('The first arty desires to terminate the tenancy for any reason or the second arty desires to vacate the said Flat can be terminated by ONE month notice on either side 6',{'entities':
  [(135, 138, 'notice')]}),
 ('This Agreement of Tenancy is made and executed at Bangalore on this the 1st day of February 2008 by and between Mr T RADHA KRISHNAN',{'entities':
  [(72, 96, 'sdate'), (112, 131, 'fname')]}),
 ('1st floor Hemalaya Golf view layout Murugeshpalya Bangalore-17 Hereinafter referred to as the OWNER of the one part and Mr AIilllJIT BHARADWAJ C/oS',{'entities':
  [(120, 142, 'sname')]}),
 ('The tenant shall pay a monthly rent of 3 500/- on or before 5th of every month Including Maintenance charges water Electricity 2',{'entities':
  [(39, 46, 'rent')]}),
 ('The tenancy shall be in force for a period of Eleven Months commencing from 1st day of February 2008 and the month of tenancy being',{'entities':
  [(46, 52, 'dur'), (76, 100, 'sdate')]}),
 ('It is hereby agreed that One month notice on either side is required for the termination of the tenancy 10',{'entities':
  [(25, 28, 'notice')]}),
 ('After the expiry of 11 months the tenant is herein agreed to pay 5 of enhancement of the existing rent',{'entities':
  [(20, 22, 'dur')]}),
 ('THIS RENTAL AGREEMENT is made on this the day of 2008 at Chennai THIS RENTAL AGREEMENT is made on this the March 22nd day 2013 at Chennai BETWEEN Mr Kamal aged about 32 years son of Appar Sundar residing at Pursawalkam Kosapt Chennai-12 hereinafter called the LESSOR which term shall mean and include whatever the context so admits and permits his legal heirs legal representative executors administrators and assigns of ONE PART and represented by his father/mother -POA agent Appar aged about 62 years residing at Pursawalkam Chennai 600 012',{'entities':
  [(107, 126, 'sdate'), (146, 154, 'fname')]}),
 ('AND Mr V Arun Kumar S/O of Mr Veda Chalam hereinafter called the LESSEE which term shall mean and include whatever the context so admits and permits his legal heirs legal representative executors administrators and assigns of OTHER PART',{'entities':
  [(4, 19, 'sname')]}),
 ('THE LESSEE has approached the LESSOR to demise the SCHEULE mentioned FLAT on Monthly rental basis for Residential purpose for a period of 12 months on the terms and conditions hereinafter mentioned and the LESSOR has also herby agreed to demise the SCHEDULE mentioned FLAT to the LESSEE on a monthly rental on the following terms and condition',{'entities':
  [(138, 140, 'dur')]}),
 ('The Lease shall be initially for a period of 12 months commencing from 23 03 13 and shall expire on the 23 03 14 upon completion of 12 months 2',{'entities':
  [(45, 54, 'dur'), (71, 79, 'sdate'), (104, 112, 'edate')]}),
 ('The LESSEE has agreed to pay the Monthly Amenity charges on or before 5th day of the every succeeding month for the demised property more fully described in the SCHEDULE hereunder in the following manner 15000 00 per month towards Rent 100000 as one time interest free deposit to be paid by the LESSEE at the time of taking possession of the premises and refunded by the LESSOR at the time of receiving vacant possession of the premises Refer Point 16 for details 4 Apart from the monthly rent the LESSEE shall pay Maintenance Charges 1000 per month directly paid to Mr Kamal Electricity Consumption charges and Water Charges 500/month if any to the authorities concerned',{'entities':
  [(204, 212, 'rent')]}),
 ('Either The LESSOR or the LESSEE may terminate the Lease agreement by giving written notice 2 months in advance',{'entities':
  [(91, 92, 'notice')]}),
 ('THIS DEED OF RENTAL AGREEMENT executed at Chennai this 19th day of August 2013 between Mrs M Geetha W/',{'entities':
  [(55, 78, 'sdate'), (87, 99, 'fname')]}),
 ('Siruvani Traders Private Limited Represented by Mr J L Saravana aged about 33 years S/o M S Jambulingam residing at No 13/5 Janakiraman Street Muthurangam Block Jafferkhanpet Ashok Nagar Chennai - 600 083 hereinafter called the tenant which expression shall wherever the context so requires be deemed to mean and include his successors- in- interest and assignees of the other part witnesses as follows -2-',{'entities':
  [(0, 32, 'sname')]}),
 ('Whereas Mrs M Geetha is the absolute owner of the property at No 62-D Saravana Flats 1st Floor Pinagapani Street Devaki Nagar Kundrathur Chennai - 600 069 measuring 250 sq ft and in possession and enjoyment of the same Whereas the tenant has approached the owner to take on rent the First Floor premises of the building referred in the schedule hereunder for running their business activities and the owner has also agreed for the same WHEREAS the parties herein mentioned felt expedient to reduce the terms and conditions of the rental agreement into writing THIS RENTAL AGREEMENT',{'entities':
  [(8, 20, 'fname')]}),
 ('That in pursuance of the above said offer and acceptance and in consideration of a monthly rent of 2500 /-',{'entities':
  [(99, 106, 'rent')]}),
 ('This Rental Agreement shall be for a period of 11 Months commencing from August 2014 2',{'entities':
  [(47, 49, 'dur'), (73, 86, 'sdate')]}),
 ('The agreed rent of 2500/- has to be paid on or before 5th of every month 3 While the owner shall bear the property tax water tax in respect of the Rented Flat the tenant shall bear Electricity Maintenance and Water Charges 4',{'entities':
  [(19, 25, 'rent')]}),
 ('Notwithstanding anything contained herein either party can terminate this lease by serving the other party a notice in writing 4 months in advance 8',{'entities':
  [(127, 128, 'notice')]}),
 ('The rental agreement can be renewed once in every 11 months with the mutual consent of the parties to this agreement IN WITNESS',{'entities':
  [(50, 52, 'dur')]}),
 ('This Rental Agreement or Residential Lease shall evidence the complete terms and conditions under which the parties whose signatures appear below have agreed Landlord SUHAIMI BIN AG AMAN AND NOOR AZEENAH',{'entities':
  [(167, 203, 'fname')]}),
 ('BINTI MICHAEL shall be referred to as OWNER and Tenant /Lessee',{'entities':
  [(0, 13, 'fname')]}),
 ('MOHD KHAIRUN BIN ZAKARIA shall be referred to as RESIDENT As consideration for this agreement',{'entities':
  [(0, 24, 'sname')]}),
 ('TERMS RESIDENT agrees to pay in advance RM 450 per month on the This agreement',{'entities':
  [(40, 46, 'rent')]}),
 ('shall commence on 1st July 2014 and continue for one 1 year as leasehold',{'entities':
  [(18, 31, 'sdate'), (53, 59, 'dur')]}),
 ('other charges are to be paid at such place or method designated by the owner as follows All payments are to be made by Cash/ Fund Transfer or via Cash Deposit Machine OWNER acknowledges receipt of the First Month s rent of RM 450 and a Security Deposit of RM 900 2 months deposit and additional charges/fees for RM 350 for a total payment of RM 1700',{'entities':
  [(223, 229, 'rent')]}),
 ('Such consent if granted shall be revocable at OWNER S option upon giving a 30 day written notice RESIDENT also agrees to carry insurance deemed appropriate by OWNER to cover possible liability and damages that may be caused by such animals 6 LIQUID',{'entities':
  [(75, 77, 'notice')]}),
 ('The terms and conditions of this agreement are subject to future change by OWNER after the expiration of the agreed lease period upon 30-day written notice setting forth such change and delivered to RESIDENT',{'entities':
  [(134, 136, 'notice')]}),
 ('THE AGREEMENT IS MADE IN DUPLICATE BETWEEN NEOH GUATWAH the Landlord and the assistant CLEMENT NG CONTACT NO 0126030570 AND the Tenant NOR AID',{'entities':
  [(43, 55, 'fname')]}),
 ('The tenancy is to begin on 1st JANUARY 2014 and is RENEWAL',{'entities':
  [(27, 43, 'sdate')]}),
 ('EVERY YEAR Minimum contract period is 1 Year TERMS',{'entities': [(38, 39, 'dur')]}),
 ('The tenant agrees to pay in advance of TWO MONTHS months of RM750 per month by 25 DECEMBER 2013',{'entities':
  [(60, 65, 'rent')]}),
 ('This agreement shall commence on 1st JANUARY 2014',{'entities': [(33, 49, 'sdate')]}),
 ('This Rental Agreement is made and Executed on at Bangalore by and Between - wef 1/4/2014',{'entities':
  [(80, 88, 'sdate')]}),
 ('Rumamurthynagar Bangalore 560016 Hereinafter Called the OWNER of the One Part Ms Pottumurthi Udayalaxmi W/o Ramakr ishna Kajjam H',{'entities':
  [(78, 103, 'sname')]}),
 ('Whereas the Tenant has paid Advance amount of 40 000/- to the owner This amount shall not carry any interest and this is refundable at the time vacating If any arrears balance that charges will deducted from the advance amount at the time of vacating has agreed to pay the Rental Charges of thousand three hundred only t month regularly the Electrcity charges separately to o 4f',{'entities':
  [(291, 313, 'rent')]}),
 ('This Rental Agreement Duration for only 11 months from this date of Agreement',{'entities':
  [(40, 42, 'dur')]}),
 ('The Tenancy can be terminated by either party by giving Two Month notice to the opposite party 6',{'entities':
  [(56, 59, 'notice')]}),
 ('This Rental Agreement is made and Executed on at Bangalore by and Between - wef 1/4/2015',{'entities':
  [(80, 88, 'sdate')]}),
 ('This agreement of rent is made and executed at Bangalore on this 1st day of April 2009 01-04-2009 by and between - Mr Parasu Ram Mishra D',{'entities':
  [(65, 97, 'sdate'), (115, 135, 'fname')]}),
 ('Mr P Gangadhar Working at M G H Corporation Gold Hill Unit 169',{'entities':
  [(0, 14, 'sname')]}),
 ('And the lessor agrees to let-out the same under the following terms and conditions - 1 whereas the lessor agrees to let-out the schedule premises for a monthly rent of 7000/- per month and the lessee shall agrees to pay the same on every month 2',{'entities':
  [(168, 174, 'rent')]}),
 ('The lease will be for period of 1 l months from the date of this Agreement',{'entities':
  [(32, 35, 'dur')]}),
 ('but it can be extended by mutual consent The rent shall be enhanced Once in every 11 months 5 over the existing rent 5',{'entities':
  [(82, 84, 'dur')]}),
 ('The lease shall vacate the premises on a one month notice from the lessor similarly the lessee can vacate the premises after giving one month notice to the lessor 10 And whereas the lessor or his subordinates or agents is at full liberty to inspect the rented premises at any reasonable hours 11',{'entities':
  [(41, 44, 'notice')]}),
 ('Please write or type below this line Rental Agreement Rental agreement was executed on Two thousand fourteen April twenty first 21/04/14 Between S T Nagaveni C/o Narayan H V Resident of Vanasuma nilaya Tumkur District SS puram 8th Cross 7th Main and Mr Madhu H N residing at 8th',{'entities':
  [(87, 136, 'sdate'), (145, 157, 'fname'), (250, 258, 'sname')]}),
 ('The Tenant shall pay a monthly rent of 5 000/- on or before 5th of every English calendar month 3',{'entities':
  [(39, 46, 'rent')]}),
 ('The term of this agreement of rent commenced from 21/04/2014',{'entities':
  [(50, 60, 'sdate')]}),
 ('This agreement will remain in force a period of 05 months 4 During the term the Security deposit amount will not carry any interest and this amount fully refundable to TENANT by OWNER while termination of this agreement or rent after deducting arrears of dues of TENANT such as electricity Bill damages rent etc thereon to which both the parties in due course of the term TENANT shall pay thereon to the concerned authority as and when it falls due In addition to that TENANT himself',{'entities':
  [(48, 50, 'dur')]}),
 ('either party may be terminated this lease by giving advance one month notice to the landlord in the absence of written notice nor a word of advice from tenant',{'entities':
  [(60, 63, 'notice')]}),
 ('3 Agreed monthly rental for the use and occupancy of residential unit will be Php 3 500 00',{'entities':
  [(82, 90, 'rent')]}),
 ('This rental agreement is made and executed on this on the 27th day January 2010 by and between SRI N VIJAY KUMAR',{'entities':
  [(58, 79, 'sdate'), (95, 112, 'fname')]}),
 ('Stage Thavarekere post Banaglore-566029 Hereinafter called the LESSO R/O W N E R of the one part and SHRI DURGA SHANKAR GUPTA working at TVS COMPNAY LIMITED Permanent Address P B',{'entities':
  [(101, 125, 'sname')]}),
 ('Whereas the terms the lessor and lessee shall mean and include their respective heirs legal representatives and assigns Whereas the lessor in the sole and absolute owner of the schedule property described hereunder and whereas the lessor is intend to let out the said house to the lessee on the rental basis for a monthly rent of 9 500/- And whereas lessee is willing to take the below house and agreeing to pay the rent to the lessor regularly if the tenant vacate with in 11 month he has to pay one month rent to owner',{'entities':
  [(330, 337, 'rent')]}),
 ('This lease executed for the period of 11 month from the date ofbut it may extended by mutual consent of the of the lessor and lessee The rent will be enhanced by 5 once in 11 month on the existing rent',{'entities':
  [(38, 46, 'dur')]}),
 ('Whereas either party should give 1 month prior notice for termination of this lease',{'entities':
  [(33, 34, 'notice')]}),
 ('AGREEMENT THIS AGREEMENT made this 27TH Day of MARCH 2016 by and betw een RC AND PAUL PAULINO herein called Landlord andherein called Tenant Landlord hereby agrees to rent the premises located at 1441 VILA ST HONOLULU HI 96818 under the following terms and conditions',{'entities':
  [(35, 57, 'sdate'), (74, 93, 'fname')]}),
 ('1 AGREEMENT Tenants agree to rent the premises for a fixed term of month to month beginning APRIL 1 2016 and ending on going Upon expiration this Agreement shall become a month-to-month agreement AUTOMATICALLY UNLESS either Tenants or Owners notify the other party in writing at least 30 days prior to expiration that they do not wish this Agreement to continue on any basis',{'entities':
  [(92, 104, 'sdate'), (285, 287, 'notice')]}),
 ('2 RENT Tenant agrees to pay Landlord as base rent the sum of S800 per month due and payable monthly in advance on the 1st day of each month during the term of this agreement',{'entities':
  [(61, 65, 'rent')]}),
 ('This HOUSE LEASE/RENTAL AGREEMENT is made and executed at the City of Olongapo City Philippines this 19th day of July 2011 by and between MARITES M ALICANTE of legal age single Filipino and with residence and postal address at No 50-20th',{'entities':
  [(101, 122, 'sdate'), (138, 156, 'fname')]}),
 ('Bajac- Olongapo City hereinafter referred to as the LANDLORD -AND- JASMIN B BAKER and EDWARD RAY BAKER JR spouses of legal ages and with residence and postal address at Phase 1 Block 2-Lot 7',{'entities':
  [(67, 105, 'sname')]}),
 ('This term of this lease agreement is for THREE 3 MONTHS from 16 July 2011 to 16 October 2011 inclusive Upon its expiration this agreement may be renewed under such terms and conditions as my be mutually agreed upon by both parties written notice of intention to renew the lease shall be served to the LANDLORD not later than seven 7 days prior to the expiry date of the period herein agreed upon 3 LEASE/RENTAL RATE',{'entities':
  [(41, 48, 'dur'), (61, 73, 'sdate'), (77, 92, 'edate')]}),
 ('The monthly rental rate for the leased premises shall be in PESOS TWENTY-TWO THOUSAND P',{'entities':
  [(60, 85, 'rent')]}),
 ('This agreement is made on the 1st day of May 2016 01 05 2016 between ANAND RAJ S',{'entities':
  [(30, 60, 'sdate'), (69, 78, 'fname')]}),
 ('/o CJ Scaria aged 26 years residing at Changankariyil House Karuvanchal PO Alakode Taliparamba Kannur Pincode 670571 on the One part AND BEENA PURATHAYIL MADHAVAN W/',{'entities':
  [(137, 162, 'sname')]}),
 ('The Lessee shall pay the Lessor a monthly rent of 11500/- regularly and punctually without any delay or default on or before 5th day of every month in the English Calendar and the receipt of the same to be obtained 2',{'entities':
  [(50, 57, 'rent')]}),
 ('The period of lease shall be 11 months from 01-05-2016 on the expiry of the above period',{'entities':
  [(29, 31, 'dur'), (44, 54, 'sdate')]}),
 ('This AGREEMENT of Rent is made in Bangalore and Executed today the lstth of May 2010 BY AND',{'entities':
  [(67, 84, 'sdate')]}),
 ('BETWEEN 1 Mr Balaji R Aged about 63 years',{'entities': [(10, 21, 'fname')]}),
 ('Whereas the first party is the sole and absolute owner of the above cited / scheduled premises is hereby continued to be rented out the same to the second party which terms and conditions is as follows The lessor have received a security Deposit amount of 40 000/- from the Lessees and hereby acknowledges the receipt of the same which carries no interest but to be returned to the lessee at the time of the lessee Vacates and hands over the position The Rent is payable by the Lessees to the Lessor is a sum of 3800/- on or before 10th of every English Calendar Month',{'entities':
  [(512, 518, 'rent')]}),
 ('This agreement is in force for a period of eleven 11 months and the same may be renewed by the mutual understanding of both the Lessor and the Lessee',{'entities':
  [(50, 52, 'dur')]}),
 ('In case of either party wants back the portion or vacates the portion either must be informed within one month prior',{'entities':
  [(101, 104, 'notice')]}),
 ('THE DAY OF 1stDAY OF SEPTEMBER 2018 Page -2 BETWEEN SMT PARMESHWARI DEVI JOSHI PAN No ACWPJ1537D W/O Sri Chothmal Joshi Hindu by faith Occupation-Housewife',{'entities':
  [(11, 35, 'sdate'), (52, 78, 'fname')]}),
 ('at-Joshi Kunj opposite Akashwani 2nd Mile Sevoke road Siliguri West Bengal hereinafter called the LESSOR/FIRST PARTY which expression shall mean and include excluded by or repugnant to the context her heirs executors sucessors administrators representatives and assigns of the FIRST PART AND TECHNOCULTURE BUILDING CENTRE PRIVATE LIMITED',{'entities':
  [(292, 337, 'sname')]}),
 ('The LESSEE having requirement of same for its commercial purpose approached/requested the LESSOR to grant lease in respect of the aforesaid office premises for a period of 36 months which shall be counted from 1st September 2018 on lease and accordingly the LESSOR agreed P/3 Cont Page 3 to grant license in favor of the LESSEE in respect of the said',{'entities':
  [(172, 174, 'dur'), (210, 228, 'sdate')]}),
 ('That the lease agreement in favor of the LESSEE has been executed for a period of 36 months commencing from 1st September 2018 2 That the LESSOR shall pay to the LESSOR a monthly lease fees amounting to 23000/- per month for the period of 36 months 3 That the LESSEE shall pay the aforesaid monthly License fee in the - succeeding month accordingly on or before the 10th of English Calendar month without claiming any deduction or abatement on any account whatsoever 4 That the LESSEE shall pay the proportionate share of the common maintenance expenses amounting to 920/- only per month on time to the LESSOR 5 That the Lessee shall pay a sum of 80 000/- only as interest free refundable security deposit to the Lessor and on completion of the lease period the Lessor shall be liable to refund the security deposit without interest subject to peaceful physical handover of the aforesaid office premises by the Lessee to the Lessor 6',{'entities':
  [(82, 84, 'dur'), (108, 126, 'sdate'), (203, 210, 'rent')]}),
 ('normal wear and tear expected removing itself together with all its articles/things on completion of the License period i e on 31st August 2021 to the LESSOR 20 That the LESSOR and the LESSEE hereby agrees with each other that if the Lessor hereof decides to terminate these presents earlier than the stipulated period herein above then the Lessor shall serve a three clear months notice to the Lessee and in case if the Lessee decides to terminate then it shall serve a one month notice expressing such intention and',{'entities':
  [(362, 380, 'notice')]}),
 ('This Agreement made this 11 day of December 2008 is between PROSPER 4 LESS LLC hereinafter referred to as Management and Jose Luis Salcido owner hereinafter referred to as Resident Management rents to Resident and Resident rents from Management property located at Addressl6714 MOSSDALE BLVD LANCASTER State California Zip 93535',{'entities':
  [(60, 78, 'fname'), (121, 138, 'sname')]}),
 ('The initial term of this Agreement shall be for 700 00 Dollars per months beginning on the 15 day of DECEMBER 2008',{'entities':
  [(48, 54, 'rent'), (91, 114, 'sdate')]}),
 ('OF MARCH 2010 BETWEEN Mr P C MATHEW S/O K JOSEPH CHACKO hereinafter called the party of the first part and between Mr L GOPINATH S/o of G LAKSHMI NARISIMHAN after called the party of the second part witnesses',{'entities':
  [(22, 35, 'fname'), (115, 128, 'sname')]}),
 ('No 65 VALLUVAR ST ARUMBAKKAM CHENNAI 600106 has agreed to let the above premises to the party of the second part for residential use on a monthly rent of 9000 plus monthly maintenance expenses levied on by the association on condition of his accepting to take on hire the house with amenities provided therein and whereas the party of the second part has agreed and will occupy the premises From 1st April 2010 1',{'entities':
  [(154, 158, 'rent'), (396, 410, 'sdate')]}),
 ('The lessor hereby let on lease the house to the lessee on a monthly rent of 9000/- for a period of 11 months starting from 1st of April 2010',{'entities':
  [(76, 82, 'rent'), (99, 101, 'dur'), (123, 140, 'sdate')]}),
 ('The lease is for a period of 11 months commencing from this day 1st of April 2010 3',{'entities':
  [(29, 31, 'dur'), (64, 81, 'sdate')]}),
 ('12 Two months notice on either side shall be the condition for the lessor and lessee to terminate this lease to voluntarily vacate the premises 13',{'entities':
  [(3, 6, 'notice')]}),
 ('AGREEMENT THIS RENTAL AGREEMENT is executed at Chennai - 600 041 on 14th day of July 2010 between Mr P JohnsonRavikumar aged about 43 Years residing at',{'entities':
  [(98, 119, 'fname')]}),
 ('Plot No 10 Door No 3 Jeevanantham Street Lakshmipuram Tiruvanmiyur Chennai - 600 041 Which term shall include their heirs assigns nominees hereinafter referred to as HOUSE OWNER and Mr Saravanan BV',{'entities':
  [(182, 194, 'sname')]}),
 ('The agreement shall be for a period of 11 months beginning form 01 04 2010 and ending with 30 03 2011 2',{'entities':
  [(39, 41, 'dur'), (64, 74, 'sdate'), (91, 101, 'edate')]}),
 ('10 000/- rent should be paid according to the English calendar month and shall be payable on or before 5th of every succeeding calendar month 3',{'entities':
  [(0, 8, 'rent')]}),
 ('The House Owner and Tenant hereby agree that three months advance notice in writing is essential for the tenant for vacating the premises if he chooses to terminate this agreement during the validity period',{'entities':
  [(45, 50, 'notice')]}),
 ('This Rental Agreement made and executed at Bangalore on this 1st day April two thousand eleven 01/04/2011 between Prof K Parthasarathy S/o Late T S Krishna Iyengar aged about 75 years No 46',{'entities':
  [(114, 134, 'fname')]}),
 ('Srinivasa 2nd Main Road Hanumanthanagar Bangalore-560019 herein after called LESSOR which expression shall where-ever the context so requires or permits mean and include here heirs executors administrators and assign of the FIRST PART AND Mr Veerabrahmam Bathini S/o Mr Lingaiah Bathini aged about 28 years Sr Engineer-PSS working at Power Research and Development Consultants 5 11th cross 2nd stage West of Chord Road',{'entities':
  [(239, 262, 'sname')]}),
 ('The Lessee will pay the Lessor for the premises herein described in the schedule a monthly rent of 8 000/-',{'entities':
  [(99, 106, 'rent')]}),
 ('Both the parties agree that this lease agreement may be terminated by the Lessee by giving three months notice in writing SCHEDULE The first floor of the No 146 VHBCSL Layout 3rd cross 1st',{'entities':
  [(91, 96, 'notice')]}),
 ('This RENTAL AGREEMENT is made executed on 20 April 2011 between Mrs Asha Ramesh Mr Ramesh K N Residing at 1st Floor 82A Vysya Bank Colony ShantiniketanLayout',{'entities':
  [(42, 55, 'sdate'), (64, 93, 'fname')]}),
 ('the LESSOR which term include their heirs administrators and assignees of the ONE PART AND Mrs Sadasivuni Deepti Mr Sadasivuni Kiran native 53-38-20',{'entities':
  [(91, 132, 'sname')]}),
 ('KRM Colony Maddilapalem Vishakapatnam AP530013 henceforth called the LESSEE which term shall mean and include their heirs executors administrators legal representatives and assigns of the OTHER PART witnessed as follows WHEREAS the LESSOR at the request of the LESSEE agreed to let out the entire 2nd floorpremise except the front room mentioned in the schedule below on a monthly rent of 5500/- Five thousand five hundred',{'entities':
  [(389, 395, 'rent')]}),
 ('only The LESSEE shall pay the monthly rent on or before 5th of every English calendarmonth The LESSEE has deposited with the LESSOR a sum of 55 000/- Fifty five thousand only as Non Interest Bearing Security Deposit and the LESSOR have acknowledged with recept The LESSOR will return the deposit to the LESSEES on the termination of rent agreement less dues if any The rent agreement shall be in force for a period of 11 from the date of Tenancy',{'entities':
  [(418, 420, 'dur')]}),
 ('It is hereby agreed that 2 Two months notice is required on either side for the termination of this Rental Agreement and',{'entities':
  [(25, 30, 'notice')]}),
 ('This agreement of tenancy is made and exceuted at chennai on this 5th day of JUNE 2009 between B Boobalan aged about 40 years residing at 5/54 Nethaji Street Okkiyam Pettai PTC Quarters Chennai - 600097 Herein after referred to as the OWNER of one part and Mr CHEBOLU VSR DIKSHITULU aged about 22 years residig at 5/54 Nethaji Street Okkiyam Pettai PTC Quarters Chennai - 600097 Herein after referred as the TENANT in consideration of the rent and condition stated NOW THIS AGREEMENT WITNESSETH',{'entities':
  [(66, 86, 'sdate'), (95, 105, 'fname'), (257, 282, 'sname')]}),
 ('The tenant shall pay a monthly rent of 6500/- from the date of occupation on or before 10th of every month 2',{'entities':
  [(39, 45, 'rent')]}),
 ('The said sum shall not carry any interest but will be refunded in full to the tenant on the termination of the tenancy and the Owner has acknowledged the same 3 The tenancy shall be enforced for a period of 11 months from this date of agreement and the month of tenancy being',{'entities':
  [(207, 209, 'dur')]}),
 ('It is hereby agreed that minimum one month notice on either side is required for the termination of the tenancy 8',{'entities':
  [(33, 36, 'notice')]}),
 ('This Rental Agreement is made and executed on 23 rd day of May',{'entities':
  [(46, 62, 'sdate')]}),
 ('1 M V Srinivas M V Madhumathi aged 54 years 42 Years residing at No 40 7th Cross Shivanandanagar Moodalapalya Bangalore - 560072 Here in after referred to as the OWNER of the first Party on the first part and 2 M V',{'entities':
  [(2, 29, 'fname')]}),
 ('The Tenant agrees to pay a monthly rent of 4 200/- to the owner on or before 5th of every month without fail',{'entities':
  [(43, 50, 'rent')]}),
 ('so the owner is empowered to deduct the cost of the same from The security deposit So paid by the tenant to the owner 9 That the duration of this Agreement is for a period of 11 Months subject to succeeding English calendar month 10',{'entities':
  [(175, 177, 'dur')]}),
 ('M V Srinivas M V Madhumathi OWNER 2',{'entities': [(0, 27, 'fname')]}),
 ('M V Thirumalesh TENANT',{'entities': [(0, 15, 'sname')]}),
 ('This Agreement is entered into Date-7/2/08 between Procall Private Limited having its Corporate Office at Procall Private Limited F-6 First floor Triveni Commercial Complex Shiekh Sarai Phase-1 New Delhi-1100 17 hereinafter referred to as Procall which expression shall unless repugnant to the context or meaning thereof',{'entities':
  [(31, 42, 'sdate'), (51, 74, 'fname')]}),
 ('includes its successors associates business affiliates subsidiaries and assigns of the one part and Address of the Party M/s Telelogic India Pvt Ltd 72 Salarpuria pearl civil station Residency road Bangalore 560025hereinafter referred to as User which expression shall unless repugnant to the context or meaning thereof',{'entities':
  [(121, 148, 'sname')]}),
 ('This will be deposited in the bank by Procall Pvt Ltd and shall be returned to the user on the expiry of the rental period after they return the radios Accessories and any other material belonging to Procall Pvt Ltd Which they have taken on rent in good order and condition 2 RENTAL PERIOD The User agrees to rent radios from Procall for a minimum period of 12 months The start date of Rental period will be the date from which Radios will be activated 3 SCHEME OF RENTAL',{'entities':
  [(358, 360, 'dur')]}),
 ('The rentals will be charged 1200/- per unit per month payable monthly in advance b Service Tax will be charged 12 36 Incase of any new Taxes are incorporated by Government of India or State Government the same would be communicated separately e',{'entities':
  [(28, 34, 'rent')]}),
 ('All spare batteries and other related accessories will be purchased by Telelogic India Pvt Ltd directly from the Motorola Authorized Repair Centre for the purpose of using Radios on rental 7 COMPLETION On expiry of the rental period or termination of the agreement the User shall immediately handover the possession of the Radios to Procall Private Limited along with the cheque of rental usage period 8 TERMINATION On Completion of 12 months paid billing either party can exit from the agreement by giving a 30 days notice period In case',{'entities':
  [(509, 511, 'notice')]})]

In [9]:
dataframe['content'] = dataframe['content'].astype('str')

In [10]:
dataframe.to_csv('validation_data_to_share.csv')

In [160]:
spacy_sentences = []
for t in train_data[125:]:
    doc = nlp(t[0])
    tags = biluo_tags_from_offsets(doc, t[1]['entities'])
    ner_info = list(zip(doc, tags))
    tokens = []
    for n, i in enumerate(ner_info):
        token = {"head" : 0,
        "dep" : "",
        "tag" : "",
        "orth" : i[0].string,
        "ner" : i[1],
        "id" : n}
        tokens.append(token)
    spacy_sentences.append(tokens)

In [161]:
with open('dev_data.json', 'w') as outfile:
    json.dump(spacy_sentences, outfile)